In [61]:
import pandas as pd
import kagglehub

In [62]:
path = kagglehub.dataset_download("jonathanpilafas/2024-march-madness-statistical-analysis")
print("Path to dataset files:", path)

Path to dataset files: /Users/pierrequereuil/.cache/kagglehub/datasets/jonathanpilafas/2024-march-madness-statistical-analysis/versions/26


In [63]:
rename_cols = {'Adjusted Temo': 'Adjusted Tempo'}
df = pd.read_csv(path + '/DEV _ March Madness.csv').rename(columns=rename_cols)

In [64]:
winners = df[df["Tournament Winner?"] == "Yes"]

winners[["Mapped ESPN Team Name", "Full Team Name"]].head()

,Mapped ESPN Team Name,Full Team Name
4,UConn,UConn Huskies
6,Kansas,Kansas Jayhawks
9,Virginia,Virginia Cavaliers
10,Baylor,Baylor Bears
11,Villanova,Villanova Wildcats


In [65]:
logos = pd.read_csv("https://raw.githubusercontent.com/IvoVillanueva/ncaa_logos/main/logos_2.csv")

logos[["ncaa_name", "reference_name", "name"]].head(n=10)

,ncaa_name,reference_name,name
0,A&M-Corpus Christi,A&M-Corpus Christi,A&M-Corpus Christi
1,Abilene Christian,Abilene Christian,Abilene Christian Wildcats
2,Air Force,Air Force,Air Force
3,Akron,Akron,Akron Zips
4,Alabama A&M,Alabama A&M,Alabama A&M
5,Alabama,Alabama,Alabama Crimson Tide
6,Alabama St.,Alabama State,Alabama State Hornets
7,Albany (NY),Albany (NY),Albany (NY)
8,Alcorn,Alcorn State,Alcorn
9,American,American,American Eagles


In [66]:
logos = pd.read_csv("logos.csv")

mapped_merge = df.merge(logos, how="left", left_on="Mapped ESPN Team Name", right_on="reference_name")
full_merge = df.merge(logos, how="left", left_on="Full Team Name", right_on="name")
last_merge = df.merge(logos, how="left", left_on="Mapped ESPN Team Name", right_on="name")

df[["Full Team Name", "Mapped ESPN Team Name"]].to_csv("df.csv")

merged = df.copy()
merged["logos"] = mapped_merge["logos"].combine_first(full_merge["logos"]).combine_first(last_merge["logos"])

merged[merged["logos"].isna()]["Full Team Name"].drop_duplicates()

1510                      IU Indy Jaguars
3127                   St. Thomas Tommies
3610               Utah Tech Trailblazers
4241                                  NaN
4585                        Queens Royals
6437    Southern Indiana Screaming Eagles
6796                    Le Moyne Dolphins
6859                 East Texas A&M Lions
7405                   Stonehill Skyhawks
7459                     Lindenwood Lions
7814                    Mercyhurst Lakers
7907                  West Georgia Wolves
Name: Full Team Name, dtype: object

In [67]:
import sqlite3

con = sqlite3.connect("venncbb/prisma/dev.db")

merged.to_sql("Team", con, index_label = "id", if_exists="replace")

8315